In [ ]:
//SVMomp.h

#ifndef SVMomp_H
#define SVMomp_H

#include <vector>
#include <string>


#include "Punto.h"

class SVMomp {
private:
    std::vector<float> weights;
    float bias;
    float learningRate;
    float regularizationParam;

public:
    SVMomp(float learningRate, float regularizationParam, int numCaracteristicas);

    void entrenar(const std::vector<Punto>& puntos, int epochs, bool mostrarProgreso = false);
    int clasificar(const Punto& puntoNuevo) const;
    void imprimirParametros(const std::vector<std::string>& etiquetas, bool mostarResumen = false) const;
};

#endif


In [ ]:
//SVMomp.cpp

// compilar: g++ -c SVMopemp.cpp -fopenmp

#include "SVMomp.h"
#include <iostream>
#include <iomanip>
#include <chrono>
#include <stdexcept>
#include <cmath>
#include <ctime>  // Para std::chrono::milliseconds
#include <sys/resource.h> // Para obtener el uso de memoria
#include <omp.h>

// Secuencia de escape para colores ANSI
#define ANSI_COLOR_GREEN   "\x1b[32m"
#define ANSI_COLOR_RESET   "\x1b[0m"

SVMomp::SVMomp(float learningRate, float regularizationParam, int numCaracteristicas)
    : learningRate(learningRate), regularizationParam(regularizationParam), bias(0.0) {
    if (numCaracteristicas <= 0) {
        throw std::invalid_argument("El número de características debe ser mayor que 0.");
    }
    weights = std::vector<float>(numCaracteristicas, 0.0); // Inicializar pesos para las características
}

void SVMomp::entrenar(const std::vector<Punto>& puntos, int epochs, bool mostrarProgreso) {
    #pragma omp parallel default(none) shared(puntos, epochs, weights, bias, learningRate, regularizationParam)
        //Directiva #pragma omp parallel: 
        //Esta directiva crea un equipo de subprocesos (threads) y define una región paralela. 
        //Cada hilo en el equipo ejecutará el bloque de código dentro de esta región paralela.
        //
        //default(none) para forzar la especificación explícita de las variables compartidas y privadas, 
        //shared para especificar las variables compartidas

    if (epochs <= 0) {
        throw std::invalid_argument("El número de épocas debe ser mayor que 0.");
    }
    if (puntos.empty()) {
        throw std::invalid_argument("El conjunto de puntos no debe estar vacío.");
    }

    auto start = std::chrono::high_resolution_clock::now(); // Marca de tiempo de inicio

    if (mostrarProgreso) {
        std::cout << "ENTRENANDO EL MODELO" << "\n--------------------" << std::endl;
    }
    
    //ajustar el modelo por iteraciones
    for (int epoch = 0; epoch < epochs; ++epoch) {
        int correct = 0; // Para contar el número de clasificaciones correctas

        //- - - - - [ OPERACIONES MATETICAS DEL MODELO ] - - - - - - - - - - 
        #pragma omp for
                //Directiva #pragma omp for: 
                //Esta directiva se utiliza para distribuir un bucle entre los hilos en la región paralela. 
                //Cada hilo ejecutará una porción del bucle de forma concurrente.
        for (size_t i = 0; i < puntos.size(); ++i) {
            const auto& punto = puntos[i];
            // Calcular x.w + b
            float decision = 0.0;
            #pragma omp simd reduction(+:decision)
                    //Directiva #pragma omp simd reduction(+:variable): 
                    //Esta directiva se utiliza para vectorizar el bucle y realizar la reducción de la variable especificada. 
                    //En el código, se utiliza para calcular la suma de las decisiones en paralelo.
            for (size_t j = 0; j < punto.caracteristicas.size(); ++j) {
                decision += punto.caracteristicas[j] * weights[j];
            }
            decision += bias;

            // Verificar si se viola la condición de margen
            if (punto.clase * decision <= 0) {
                #pragma omp critical
                        //Directiva #pragma omp critical: 
                        //Esta directiva se utiliza para proteger una sección crítica del código, asegurando que solo un hilo pueda ejecutarla a la vez. 
                        //En el código proporcionado, se utiliza para garantizar que la impresión de la información de progreso del entrenamiento (cout) se realice de forma segura, evitando la escritura simultánea de múltiples hilos en la salida estándar.
                // Actualizar pesos y bias
                for (size_t j = 0; j < weights.size(); ++j) {
                    weights[j] += learningRate * (punto.clase * punto.caracteristicas[j] - regularizationParam * weights[j]);
                }
                bias += learningRate * punto.clase;
            } else {
                correct++;
            }
        //- - - - - - - - - - - - - - - - - - - - - - - -  

            if (mostrarProgreso) {
                // Mostrar progreso
                int interval = puntos.size() / 10 == 0 ? 1 : puntos.size() / 10; // Evitar división por cero
                if (i % interval == 0 || i == puntos.size() - 1) { // Muestra progreso cada 10% y al final
                    int progress = static_cast<int>(std::round((i + 1) * 100.0 / puntos.size()));
                    int barWidth = 20;
                    int pos = barWidth * progress / 100;

                    // Obtener el uso de memoria
                    struct rusage usage;
                    getrusage(RUSAGE_SELF, &usage);
                    long memoryUsage = usage.ru_maxrss; // Uso máximo de memoria residente

                    std::cout << "\rEpoch " << epoch + 1 << "/" << epochs 
                              << " [";
                    for (int k = 0; k < barWidth; ++k) {
                        if (k < pos) std::cout << "=";
                        else if (k == pos) std::cout << ">";
                        else std::cout << " ";
                    }
                    std::cout << "] " << std::setw(3) << progress << "% "
                              << "Correctos: " << correct << " de " << puntos.size()
                              << std::flush;
                }
            }
        }
        // Fin de la época
        if (mostrarProgreso) {
            std::cout << "\rEpoch " << epoch + 1 << "/" << epochs << " [====================] " << ANSI_COLOR_GREEN << "100% Correctos: " << correct << " de " << puntos.size() <<" (NO violan la condición de margen)" << ANSI_COLOR_RESET <<" { " << "Sesgo (Bias): " << bias <<" }" << std::endl;
        }
    }

    auto end = std::chrono::high_resolution_clock::now(); // Marca de tiempo de fin
    std::chrono::duration<double> duration = end - start; // Duración del entrenamiento

    // Obtener el uso de memoria final
    struct rusage usage;
    getrusage(RUSAGE_SELF, &usage);
    long memoryUsage = usage.ru_maxrss; // Uso máximo de memoria residente

    if (mostrarProgreso) {
        std::cout << "\nRESUMEN CONSUMO ENTRENAMIENTO\n";
        std::cout << " - Entrenamiento completo en " << duration.count() << " segundos." << std::endl;
        std::cout << " - Uso máximo de memoria: " << memoryUsage << " KB" << std::endl << std::endl;
    } else {
        std::cout << "Modelo entrenado " << ANSI_COLOR_GREEN <<"OK." << ANSI_COLOR_RESET << std::endl << std::endl;
    }
}

int SVMomp::clasificar(const Punto& puntoNuevo) const {
    float decision = 0.0;
    for (size_t i = 0; i < puntoNuevo.caracteristicas.size(); ++i) {
        decision += puntoNuevo.caracteristicas[i] * weights[i];
    }
    decision += bias;
    return (decision >= 0) ? 1 : -1;
}

void SVMomp::imprimirParametros(const std::vector<std::string>& etiquetas, bool mostarResumen) const {
    if (mostarResumen){
        std::cout << "\nRESUMEN DE LOS PARÁMETROS" << "\n----------------------------" << std::endl;
        for (size_t i = 0; i < weights.size(); ++i) {
            std::cout << " w" << i << " {" << etiquetas[i] << "}: " << weights[i] << std::endl;
        }
        std::cout << " B: " << bias << std::endl << std::endl;
    } else {
        std::cout << "Modelo entrenado " << ANSI_COLOR_GREEN <<"OK." << ANSI_COLOR_RESET << std::endl << std::endl;
    }
}


In [ ]:
//main_omp.cpp

#include <iostream>
#include <vector>
#include <string>
#include <limits>
#include "Datos.h"
#include "SVMomp.h"


// Colores para la salida
const std::string rojo = "\033[41;30m";
const std::string verde = "\033[42;30m";
const std::string reset = "\033[0m";

void ingresarYClasificar(SVMomp& svm, const std::vector<std::string>& etiquetas);
void parsear_argumentos(int argc, char* argv[], std::string& fuente_datos, float& tasa_aprendizaje, float& overfitting, int& epocas, bool& echo);
 
int main(int argc, char* argv[]) {
    // Variables para almacenar los valores de los argumentos y valores por defecto 
    std::string fuente_datos;  //dataser con datos de conocimiento
    float tasa_aprendizaje = 0.001; //control de la velocidad de aprendizaje de modelo
    float overfitting = 0.001; //parámetro de regularización
    int epocas = 5;  //iteraciones del modelo
    bool echo = false;  //Bandera para ver o no el entrenamiento

    try {
        // Parsear argumentos de línea de comandos
        parsear_argumentos(argc, argv, fuente_datos, tasa_aprendizaje, overfitting, epocas, echo);

        // Imprimir información del programa
        std::cout << "Ejecución del programa...\n\n"; 
        std::cout << "DATOS DE LA INSTANCIA DEL PROGRAMA" << "\n----------------------------------\n";
        std::cout << " - Fuente de datos: " << fuente_datos << std::endl;
        std::cout << " - Tasa de aprendizaje: " << tasa_aprendizaje << std::endl;
        std::cout << " - Overfitting: " << overfitting << std::endl;
        std::cout << " - Epocas: " << epocas << std::endl;
        std::cout << " - Echo: " << (echo ? "true" : "false") << std::endl << std::endl;

        Datos lector;
        std::vector<Punto> puntos = lector.leerDatosCSV(fuente_datos);

        // Verificar que se leyeron datos correctamente
        if (puntos.empty() || lector.etiquetas.empty()) {
            std::cerr << rojo << "No se pudieron leer los datos del archivo CSV." << reset << std::endl;
            return 1;
        }

        // Crear el modelo SVM
        //float tasaAprendizaje = 0.001; //control de la velocidad de aprendizaje de modelo
        //float overfitting = 0.01; //parámetro de regularización
        int numCaracteristicas = puntos[0].caracteristicas.size(); //cantidad de variables o atributos que describen la instancia en un conjunto de datos.
        SVMomp svm(tasa_aprendizaje, overfitting, numCaracteristicas);

        // Entrenar el modelo SVM
        svm.entrenar(puntos, epocas, echo);

        // Mostrar los pesos y el sesgo del hiperplano separador
        svm.imprimirParametros(lector.etiquetas, echo);

        // Ingresar y clasificar un nuevo punto de datos
        ingresarYClasificar(svm, lector.etiquetas);

    } catch (const std::runtime_error& e) {
        std::cerr << "Error: " << e.what() << std::endl;
        return 1;
    }

    return 0;
}

//=============================================================================
void ingresarYClasificar(SVMomp& svm, const std::vector<std::string>& etiquetas) {
    std::cout << "\nPRUEBAS\n" << "-------\n";

    bool otraClasificacion = true;
    while (otraClasificacion) {
        // Ingresar las características del nuevo punto
        std::cout << "Ingrese las características del nuevo punto:" << std::endl;
        std::vector<float> nuevasCaracteristicas(etiquetas.size());

        for (size_t i = 0; i < etiquetas.size(); ++i) {
            std::cout << "  " << etiquetas[i] << ": ";
            while (!(std::cin >> nuevasCaracteristicas[i])) {
                std::cin.clear();
                std::cin.ignore(std::numeric_limits<std::streamsize>::max(), '\n');
                std::cout << "         " << "\033[41;30m" << "Valor inválido. (Intente de nuevo!)"<< "\033[0m " << std::endl << "  " << etiquetas[i] << ": ";
            }
        }

        Punto puntoNuevo(nuevasCaracteristicas, 0); // La clase es irrelevante para la predicción
        int clasePredicha = svm.clasificar(puntoNuevo);

        // Colores para la salida
        const std::string rojo = "\033[41;30m";
        const std::string verde = "\033[42;30m";
        const std::string reset = "\033[0m";

        if (clasePredicha == 1) {
            std::cout << "Clase predicha para el nuevo punto: " << rojo << "Maligno" << reset << std::endl;
        } else {
            std::cout << "Clase predicha para el nuevo punto: " << verde << "Benigno" << reset << std::endl;
        }
        std::cout << "\033[0m";

        // Preguntar si el usuario desea otra clasificación
        char otraClasificacionChar;
        std::cout << "\n¿Desea otra clasificación? ("<<verde<<" [si] s "<<reset<<" - "<<rojo<<" [no] n "<<reset<<"): ";
        std::cin >> otraClasificacionChar;

        // Convertir la respuesta del usuario a un valor booleano
        otraClasificacion = !(otraClasificacionChar == 'n' || otraClasificacionChar == 'N');
    }
}

//============================================================================================================
// Función para parsear argumentos de línea de comandos
void parsear_argumentos(int argc, char* argv[], std::string& fuente_datos, float& tasa_aprendizaje, float& overfitting, int& epocas, bool& echo) {
  // Variables para almacenar argumentos
  std::string argumento;

  // Procesar cada argumento
  for (int i = 1; i < argc; ++i) {
    argumento = argv[i];

    // Fuente de datos
    if (argumento == "--fuente_datos") {
      if (i + 1 < argc) {
        fuente_datos = argv[i + 1];
        ++i;
      } else {
        throw std::runtime_error("Falta el argumento para --fuente_datos");
      }
    }

    // Tasa de aprendizaje
    else if (argumento == "--tasa_aprendizaje") {
      if (i + 1 < argc) {
        try {
          tasa_aprendizaje = std::stof(argv[i + 1]);
        } catch (const std::exception& e) {
          throw std::runtime_error("Valor inválido para --tasa_aprendizaje: " + std::string(e.what()));
        }
        ++i;
      } else {
        throw std::runtime_error("Falta el argumento para --tasa_aprendizaje");
      }
    }

    // Overfitting
    else if (argumento == "--overfitting") {
      if (i + 1 < argc) {
        try {
          overfitting = std::stof(argv[i + 1]);
        } catch (const std::exception& e) {
          throw std::runtime_error("Valor inválido para --overfitting: " + std::string(e.what()));
        }
        ++i;
      } else {
        throw std::runtime_error("Falta el argumento para --overfitting");
      }
    }

    // Epocas
    else if (argumento == "--epocas") {
      if (i + 1 < argc) {
        try {
          epocas = std::stoi(argv[i + 1]);
        } catch (const std::exception& e) {
          throw std::runtime_error("Valor inválido para --epocas: " + std::string(e.what()));
        }
        ++i;
      } else {
        throw std::runtime_error("Falta el argumento para --epocas");
      }
    }

    // Echo
    else if (argumento == "--echo") {
      echo = true;
    }

    // Argumento no reconocido
    else {
      throw std::runtime_error("Argumento no reconocido: " + argumento);
    }
  }
}

In [ ]:
!mkdir -p ../out_omp

!g++ -c ../src_omp/Punto.cpp -o ../out_omp/Punto.o -fopenmp
!g++ -c ../src_omp/Datos.cpp -o ../out_omp/Datos.o -fopenmp
!g++ -c ../src_omp/SVMomp.cpp -o ../out_omp/SVMomp.o -fopenmp

!g++ ../src_omp/main_omp.cpp ../out_omp/Punto.o ../out_omp/Datos.o ../out_omp/SVMomp.o -o ../out_omp/demo -fopenmp

!../out_omp/demo --fuente_datos ../../../dataset/datos_completos.csv --tasa_aprendizaje 0.0001 --overfitting 0.0001 --epocas 10 --echo